<a href="https://colab.research.google.com/github/juanignaciogiacobbe/7506R-2C2023-GRUPO15/blob/main/7506R_TP1_GRUPO15_CHP3_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 1: Reservas de Hotel</center>
### <center> Grupo número 15</center>
### <center> Giacobbe, Juan Ignacio; Olaran, Sebastian; Ayala, Tomás</center>

#Checkpoint 3: Ensambles

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las librerías que vamos a utilizar

In [9]:
!pip install dtreeviz

#Para análisis de los datos
import pandas as pd
import numpy as np


#Para visualizaciones
import seaborn as sns
from matplotlib import pyplot as plt
import dtreeviz
import warnings
warnings.filterwarnings('always')

#Para crear nuestros modelos predictivos
import sklearn as sk
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import DistanceMetric


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


Cargamos los archivos csv de datos en dos DataFrames de Pandas(importamos para el dataset de train el dataset que nos quedó como resultado en el checkpoint 2)

In [3]:
ds_hoteles_test = pd.read_csv('./ds_hoteles_test_checkpoint_3.csv')
ds_hoteles_train = pd.read_csv('./ds_hoteles_train_checkpoint_3.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


##Breve resumen del dataset
Comenzamos haciendo un breve repaso de las características del dataset a analizar

In [4]:
ds_hoteles_train.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


(60897, 47)

Tenemos un dataset con 60897 registros de reservas de hotel, con 32 columnas correspondientes a variables que describen las reservas(por ejemplo, tenemos una variable que indica la cantidad de adultos que tenemos por reserva).


Veamos los primeros dos registros del dataset

In [5]:
ds_hoteles_train.head(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreCl

,Unnamed: 0.1,Unnamed: 0,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,market_segment_Undefined,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_Non Refund,deposit_type_Refundable
0,0,0,49,2016,0,37,5,1,2,1,...,0,0,0,1,0,0,1,0,0,0
1,1,1,4,2015,1,44,31,0,1,2,...,0,1,0,0,0,0,1,0,0,0


Veamos las variables que hay en el dataset

In [6]:
ds_hoteles_train.columns.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


['Unnamed: 0.1',
 'Unnamed: 0',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'country',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'agent',
 'company',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'is_canceled',
 'hotel_Resort Hotel',
 'meal_FB',
 'meal_HB',
 'meal_SC',
 'meal_Undefined',
 'market_segment_Complementary',
 'market_segment_Corporate',
 'market_segment_Direct',
 'market_segment_Groups',
 'market_segment_Offline TA/TO',
 'market_segment_Online TA',
 'market_segment_Undefined',
 'distribution_channel_Direct',
 'distribution_channel_GDS',
 'distribution_channel_TA/TO',
 'distribution_channel_Undefined',
 'customer_type_Group',
 'customer_type_Transient',
 'cus

**Descripción de cada campo**



*   hotel(categórica): indica el tipo de hotel en el que se realizó la reserva.
*   lead_time(cualitativa): indica el tiempo desde que se reservó hasta que la persona llegó al hotel.
*   arrival_date_year/month/week_number/day_of_month: indican la fecha de llegada.
*   stays_in_weekend/week_nights: indican las noches que están reservadas.
*   adults, children, babies: indican la cantidad de adultos, niños y bebés respectivamente.
*   meal: indica el tipo de comida que incluye la reserva.
*   country: el país destino de la reserva.
*   market_segment: Segmento del mercado al cual está destinada la reserva.
*   distribution_chanel: Canal de distribución de reservas
*   is_repeated_guest: Indica si el cliente ya ha tenido experiencia reservando.
*   previous_cancellations/ previous_bookings_not_canceled: Indican la cantidad de reservas canceladas y no canceladas de un cliente.
*   reserved_room_type/ assigned_room_type: Tipo de habitación reservada y asignada respectivamente.
*   booking_changes: Cambios que hubieron en el booking.
*   deposit_type: Tipo de depósito que tuvo la reserva.
*   agent: Agente de viajes al que está asignada la reserva.
*   company: Empresa a cargo del booking de la reserva.
*   days_in_waiting_list: Días en que la reserva estuvo en lista de espera.
*   customer_type: Tipo de cliente que tuvo la reserva.
*   adr: Precio promedio de alquiler por día.
*   required_car_parking_spaces: Cantidad de espacios de estacionamiento requeridos.
*   total_of_special_requests: Cantidad de invitados.



---

##Construcción de ensambles

Enumeramos los features que utilizaremos para nuestros modelos predictivos

In [7]:
features = ['hotel_Resort Hotel', 'meal_FB', 'meal_HB', 'meal_SC', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights',
 'adults', 'children', 'babies', 'country', 'meal_Undefined', 'market_segment_Complementary', 'market_segment_Corporate', 'market_segment_Direct', 'market_segment_Groups', 'market_segment_Offline TA/TO',
'market_segment_Online TA', 'market_segment_Undefined',
 'customer_type_Group', 'customer_type_Transient', 'customer_type_Transient-Party', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled',
  'booking_changes', 'deposit_type_Non Refund', 'deposit_type_Refundable', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
 'total_of_special_requests']

target = 'is_canceled'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


Separamos en conjuntos de train y test

In [8]:
#Separo un set de Evaluacion
from sklearn.model_selection import train_test_split

#Creo un dataset con los features que voy a usar para clasificar
ds_trabajo_x=ds_hoteles_train.drop([target], axis='columns', inplace=False)

#Creo un dataset con la variable target
ds_trabajo_y = ds_hoteles_train[target].copy()

x_train, x_test, y_train, y_test = train_test_split(ds_hoteles_train[features].values,
                                                    ds_hoteles_train[target].values,
                                                    test_size=0.2,    #20% al conjunto de test
                                                    random_state=13,  #para poder reproducir el experimento
                                                    stratify=ds_hoteles_train[target].values) #estratificado para mantener proporcion

#Verifico Cantidad de Datos en cada set
print('# Datos Entrenamiento: {}'.format(len(x_train)))
print('# Datos Prueba: {}'.format(len(x_test)))

#Verifico como quedaron balanceados
for split_name, split in zip(['% Positivos Entrenamiento','% Positivos Prueba'],[y_train,y_test]):
  print('{}: {:.3f}'.format(split_name,pd.Series(split).value_counts(normalize=True)[1]*100))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


# Datos Entrenamiento: 48717
# Datos Prueba: 12180
% Positivos Entrenamiento: 50.459
% Positivos Prueba: 50.460


###Clasificador KNN

Lo que haremos será optimizar sus hiperparámetros mediante k-fold Cross Validation

In [10]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(1,100),
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev']
             }

#Clasificador KNN
knn=KNeighborsClassifier()

#Random Search con 10 Folds y 10 iteraciones
rand = RandomizedSearchCV(knn, params_grid, cv=10, scoring='f1', n_iter=10, random_state=5)

rand.fit(x_train, y_train)
rand.cv_results_['mean_test_score']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


array([0.73040244, 0.777184  , 0.6690257 , 0.74757978, 0.74740869,
       0.78669156, 0.69491765, 0.7477069 , 0.75756273, 0.66309074])

In [11]:
#Mejores hiperparametros
print(rand.best_params_)
#Mejor métrica
print(rand.best_score_)

{'weights': 'distance', 'n_neighbors': 20, 'metric': 'manhattan', 'algorithm': 'brute'}
0.7866915589604487


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [12]:
#Mejor estimador
best_knn=rand.best_estimator_

#Predicción con mejor estimador
y_pred=best_knn.predict(x_test)

#Accuracy test
f1_score(y_test,y_pred)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


0.7887975744035745

segundo KNN

In [14]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(1,100),
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev']
             }

#Clasificador KNN
knn=KNeighborsClassifier()

#Random Search con 10 Folds y 10 iteraciones
rand = RandomizedSearchCV(knn, params_grid, cv=5, scoring='f1', n_iter=100, random_state=5)

rand.fit(x_train, y_train)
rand.cv_results_['mean_test_score']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


array([0.75513352, 0.78218996, 0.75578016, 0.75366916, 0.77471387,
       0.78147878, 0.6915248 , 0.72446397, 0.67569398, 0.78281773,
       0.72957172, 0.72546457, 0.78167783, 0.66200455, 0.78157629,
       0.78259708, 0.73426803, 0.78167545, 0.7550493 , 0.68754312,
       0.72942662, 0.70815344, 0.74404232, 0.78243582, 0.78284003,
       0.74454328, 0.69243886, 0.74410287, 0.7561692 , 0.68164568,
       0.75361071, 0.71023167, 0.68163018, 0.7028966 , 0.78197546,
       0.75502898, 0.75455055, 0.75558678, 0.78262822, 0.6693847 ,
       0.72366561, 0.66667955, 0.77564232, 0.73200863, 0.74537959,
       0.73258472, 0.78159836, 0.73376703, 0.68439881, 0.68716464,
       0.72432215, 0.77756122, 0.74454049, 0.66231638, 0.67191828,
       0.67259269, 0.68529939, 0.6802104 , 0.70269402, 0.721965  ,
       0.7462963 , 0.75544465, 0.68026793, 0.72712729, 0.68823666,
       0.68636947, 0.75553196, 0.7426937 , 0.75463616, 0.74512977,
       0.77001289, 0.66641046, 0.68286608, 0.78249513, 0.78201

In [15]:
#Mejores hiperparametros
print(rand.best_params_)
#Mejor métrica
print(rand.best_score_)

{'weights': 'distance', 'n_neighbors': 33, 'metric': 'manhattan', 'algorithm': 'kd_tree'}
0.78290768243381


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [16]:
#Mejor estimador
best_knn=rand.best_estimator_

#Predicción con mejor estimador
y_pred=best_knn.predict(x_test)

#Accuracy test
f1_score(y_test,y_pred)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


0.7896161809205287

prediccion sobre el test

In [17]:
ds_test_prediccion = ds_hoteles_test.copy()

ds_test_prediccion['market_segment_Undefined'] = 0
ds_test_prediccion['distribution_channel_Undefined'] = 0
ds_test_prediccion['agent'] = ds_test_prediccion['agent'].fillna('-1')
ds_test_prediccion['company'] = ds_test_prediccion['company'].fillna('-1')

ds_test_prediccion.drop('id', inplace=True, axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [18]:
predict = best_knn.predict(ds_test_prediccion[features])

ds_resultados=pd.DataFrame(zip(ds_hoteles_test['id'], predict),columns=['id','is_canceled'])

ds_resultados['is_canceled'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names


1    14569
0    11966
Name: is_canceled, dtype: int64

In [19]:
ds_resultados_copy_csv = ds_resultados.copy()
ds_resultados_copy_csv.set_index("id", inplace = True)
ds_resultados_copy_csv

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,is_canceled
id,
e6d0d58a-ab96-44d3-b28f-85df1cc15f1e,1
aba9aac2-4495-4645-a60e-90328713c999,1
b345b5dc-c804-4d77-a39f-1f4e65d1f1ad,0
ce1b400e-afa1-4946-bdff-004b859eb61c,1
c4fbec0d-8e2e-4db2-9d4f-a6ccb9f51214,1
...,...
636ccded-66ad-4538-8006-c0b9f84c6256,0
11e19741-bd28-4459-8ea3-184f1664000f,0
36a4c7b0-d6b1-4528-93db-311be96ec1c6,1


In [20]:
ds_resultados.to_csv("predicciones_knn1.csv", index = False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
